In [1]:
# Import Numpy & PyTorch
import numpy as np
import torch

A tensor is a number, vector, matrix or any n-dimensional array.

## **Problem Statement**

We'll create a model that predicts crop yeilds for apples (*target variable*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in different regions. 

Here's the training data:

>Temp | Rain | Humidity | Prediction
>--- | --- | --- | ---
> 73 | 67 | 43 | 56
> 91 | 88 | 64 | 81
> 87 | 134 | 58 | 119
> 102 | 43 | 37 | 22
> 69 | 96 | 70 | 103

In a **linear regression** model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
```

It means that the yield of apples is a linear or planar function of the temperature, rainfall & humidity.



**Our objective**: Find a suitable set of *weights* and *biases* using the training data, to make accurate predictions.

## Training Data
The training data can be represented using 2 matrices (inputs and targets), each with one row per observation and one column for variable.

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Target (apples)
targets = np.array([[56], 
                    [81], 
                    [119], 
                    [22], 
                    [103]], dtype='float32')

Before we build a model, we need to convert inputs and targets to PyTorch tensors.


In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Linear Regression Model (from scratch)

The *weights* and *biases* can also be represented as matrices, initialized with random values. The first row of `w` and the first element of `b` are use to predict the first target variable i.e. yield for apples, and similarly the second for oranges.

In [5]:
W = torch.rand((1,inputs.shape[1]),requires_grad=True)
b = torch.zeros(size=(inputs.shape[0],1),requires_grad=True)
W

tensor([[0.8462, 0.9300, 0.6536]], requires_grad=True)

The *model* is simply a function that performs a matrix multiplication of the input `x` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

$$
\hspace{2.5cm} X \hspace{1.1cm} \times \hspace{1.2cm} W^T \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\times
%
\left[ \begin{array}{cc}
w_{11}  \\
w_{12} \\
w_{13} 
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b\\
\end{array} \right]
$$

In [6]:
predict = lambda X,W,b : X@W.T + b

The matrix obtained by passing the input data to the model is a set of predictions for the target variables.

In [7]:
# Generate predictions
yhat = predict(inputs,W, b)

In [8]:
# Compare with targets
print(yhat)
print(targets)

tensor([[152.1891],
        [200.6766],
        [236.1526],
        [150.4864],
        [193.4220]], grad_fn=<AddBackward0>)
tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


Because we've started with random weights and biases, the model does not perform a good job of predicting the target varaibles.

## Loss Function

We can compare the predictions with the actual targets, using the following method: 
* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [9]:
# MSE loss
loss_fn = torch.nn.MSELoss()
my_loss = lambda Y,Yhat : ((Y-Yhat)**2).mean()

In [10]:
# Compute loss
loss = loss_fn(targets,yhat)


The resulting number is called the **loss**, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model. 

## Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the `loss` w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

More on autograd:  https://pytorch.org/docs/stable/autograd.html#module-torch.autograd

In [11]:
# Compute gradients
loss.backward()

The gradients are stored in the .grad property of the respective tensors.

In [12]:
# Gradients for weights
W.grad

tensor([[18979.7500, 18752.0332, 11869.5283]])

In [13]:
# Gradients for bias
b.grad

tensor([[38.4756],
        [47.8706],
        [46.8610],
        [51.3946],
        [36.1688]])

A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases. 

* If a gradient element is **postive**, 
    * **increasing** the element's value slightly will **increase** the loss.
    * **decreasing** the element's value slightly will **decrease** the loss.




* If a gradient element is **negative**,
    * **increasing** the element's value slightly will **decrease** the loss.
    * **decreasing** the element's value slightly will **increase** the loss.
    


The increase or decrease is proportional to the value of the gradient.

Finally, we'll reset the gradients to zero before moving forward, because PyTorch accumulates gradients.

In [14]:
W.grad.zero_(),b.grad.zero_()

(tensor([[0., 0., 0.]]), tensor([[0.],
         [0.],
         [0.],
         [0.],
         [0.]]))

## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [15]:
X = torch.from_numpy( np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32'))
Y = torch.from_numpy( np.array([[56], 
                    [81], 
                    [119], 
                    [22], 
                    [103]], dtype='float32'))

W = torch.rand((1,X.shape[1]),requires_grad=True)
b = torch.tensor([[0.0]],requires_grad=True)

predict = lambda X,W,b : X@W.T + b
my_loss = lambda Y,Yhat : ((Y-Yhat)**2).mean()

find_w_grad = lambda yhat,Y,X:2*(yhat-Y).T@X/len(Y)
find_b_grad = lambda yhat,Y:sum(2*(yhat-Y))/len(Y)

loss_fn = torch.nn.MSELoss()

W,b,W.grad,b.grad

(tensor([[0.9687, 0.9693, 0.7988]], requires_grad=True),
 tensor([[0.]], requires_grad=True),
 None,
 None)

In [16]:
# Generate predictions
predictions = predict(X,W,b)
predictions

tensor([[170.0025],
        [224.5677],
        [260.4884],
        [170.0381],
        [215.8040]], grad_fn=<AddBackward0>)

In [17]:
# Calculate the loss
loss = loss_fn(Y,predictions)
loss

tensor(17653.4414, grad_fn=<MseLossBackward>)

In [18]:
# Compute gradients
loss.backward()
print(W.grad,b.grad)

print(find_w_grad(predictions,Y,X),find_b_grad(predictions,Y))

tensor([[22631.8730, 22570.5527, 14268.1797]]) tensor([[263.9602]])
tensor([[22631.8750, 22570.5527, 14268.1797]], grad_fn=<DivBackward0>) tensor([263.9602], grad_fn=<DivBackward0>)


In [19]:
alpha = 1e-3  

In [20]:
# Adjust weights & reset gradients
with torch.no_grad():
    W -= alpha*(W.grad)
    b -= alpha*(b.grad)
W.grad.zero_(),b.grad.zero_()
W,b

(tensor([[-21.6632, -21.6013, -13.4694]], requires_grad=True),
 tensor([[-0.2640]], requires_grad=True))

With the new weights and biases, the model should have a lower loss.

In [21]:
new_pred = predict(X,W,b)
# Calculate loss
loss = loss_fn(Y,new_pred)
loss

tensor(20711834., grad_fn=<MseLossBackward>)

## Train for multiple epochs

To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.

In [22]:
# Train for 100 epochs
epoch =150
alpha = 1e-5
W = torch.randn((1,X.shape[1]),requires_grad=True,dtype=torch.float32)
b = torch.randn((1,1),requires_grad=True,dtype=torch.float32)

for e in range(epoch):
    yhat = predict(X,W,b)
    loss = loss_fn(Y,yhat)
    loss.backward()
    with torch.no_grad():
        W -= alpha*W.grad
        b -= alpha*b.grad
#     print(W.grad,b.grad)
    W.grad.zero_(),b.grad.zero_()
#     print(e," : ",loss.detach().numpy())
    

In [23]:
# Calculate loss
loss_fn(Y,yhat)

tensor(30.7569, grad_fn=<MseLossBackward>)

In [24]:
# Print predictions
predict(X,W,b)

tensor([[ 57.6614],
        [ 78.4151],
        [126.5187],
        [ 23.6487],
        [ 93.8058]], grad_fn=<AddBackward0>)

In [25]:
# Print targets
Y

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])